<center><img src="https://github.com/FASSt-simulation/fasst_simulation_tools/raw/main/images/site-logo.png" width=150 height=150 alt="NGEE-Arctic Logo"/></center>

# An example of reading in a plotting one variable from ELM output.

#### ----------------------------------------------------------------------------------------------------------------------

<br>

To run this script and generate the resulting ELM plots we need specific Python libraries.  This step loads those required libraries

In [1]:
import matplotlib.pyplot as plt
import xarray
import os
import ipywidgets as widgets
import glob, numpy

Next, we locate the output data and identify which sites there is output for. This code creates a dropdown menu to pick the model run you want to plot.

In [2]:
output_rootdir=os.path.expanduser('~')+'/output/cime_run_dirs/'
cases=sorted(glob.glob("%s*20TR*" % output_rootdir))
cases=[os.path.basename(x) for x in cases]
cases_dropdown = widgets.Dropdown(options=cases,
                                description='Chose Case Name:',
                                style={'description_width':'auto'},
                                layout={'width':'max-content'},
                                disabled=False)

display(cases_dropdown)

Here, we are using the combined output file over all years of the run. We read in the list of variables, units, and longer descriptions from the dataset. We use the time variable from the dataset to determine the start and end years.

In [7]:
output_file=output_rootdir + cases_dropdown.value + '/run/ELM_output.nc'

# Read in the list of variable, units, and longer descriptions from the dataset
with xarray.open_dataset(output_file) as data:
    vars=data.variables
    vars_names=list(vars.keys())
    units=[data[var].attrs.get('units',None) for var in vars_names]
    longnames=[data[var].attrs.get('long_name',None) for var in vars_names]

    description=[f'{var} ({longname}) [{unit}]' for (var,longname,unit) in zip(vars_names,units,longnames)]

    # Determine start and end year of the dataset
    data_startyear=data.time[0].item().year
    data_endyear=data.time[-1].item().year

vars_dropdown = widgets.Dropdown(options=list(zip(description,vars_names)),
                                description='Choose Variable Name',
                                style={'description_width':'auto'},
                                value='GPP',
                                layout={'width':'max-content'},
                                disabled=False)
startyear_picker = widgets.BoundedIntText(value=data_startyear, min=data_startyear, max=data_endyear, step=1,
                                          description='Choose start year',style={'description_width':'auto'})
endyear_picker = widgets.BoundedIntText(value=data_endyear, min=data_startyear, max=data_endyear, step=1,
                                          description='Choose end year',style={'description_width':'auto'})

display(vars_dropdown)
display(startyear_picker)
display(endyear_picker)

Using the fields above, you can pick the variable that you want to plot, and the start and end year of the time series to visualize.

Next, we read the variable from the dataset and select the time portion based on start and end year picked above.

In [10]:
year_start=startyear_picker.value
year_end=endyear_picker.value
var=vars_dropdown.value

with xarray.open_dataset(output_file).squeeze() as output:
    if 'time' not in output[var].dims:
        print(f'Variable {var} is not a time-series variable! Plotting will not work.')
    else:
        vardata=output[var].sel(time=slice(str(year_start),str(year_end)))
    
    if 'PFTnames' in output:
        print('Output file is separated by plant functional type')
        PFTnames=output['PFTnames']
    else:
        PFTnames=[]


Finally, we will create a figure and plot the variable. The code checks whether it is depth-defined or not. For depth-defined variables, it plots a two-dimensional time vs depth plot. Otherwise, it plots a time series. For model output that shows each plant functional type individually, it plots the plant functional types separately.

In [11]:
f,a = plt.subplots()

if 'levgrnd' in vardata.dims or 'levdcmp' in vardata.dims:
    print('Variable is depth-defined. Plotting in two dimensions')
    vardata.T.plot()
    maxdepth=2.0
    a.set_ylim(maxdepth,0.0)
    a.set_ylabel('Soil depth (m)')
elif 'PFT' in vardata.dims:
    print('Variable is defined by plant functional type. Plotting one line per PFT')
    for pftnum in vardata.PFT:
        pftname=vardata.PFTnames[pftnum]
        vardata.sel(pft=pftnum).plot(label=f'PFT: {pftname}')
    a.legend()
else:
    print('Variable is not depth-defined. Plotting line plot.')
    vardata.plot()

<center><img src="https://github.com/FASSt-simulation/fasst_simulation_tools/raw/main/images/site-logo.png" width=150 height=150 alt="NGEE-Arctic Logo"/></center>